<a href="https://colab.research.google.com/github/ramapathapati/hds5210-2023/blob/main/week06/week06_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 6 Exercises

_McKinney 6.1_

There are multiple ways to solve the problems below.  You can use any one of several approaches.  For example, you can read CSV files using Pandas or the csv module.  Your score won't depend on which modules you choose to use unless explicitly noted below, but your programming style will still matter.

### 30.1 List of Allergies

In this GitHub repository, there is a file called `allergies.json` that contains a list of patient allergies.  You will need to download this [file from here](https://raw.githubusercontent.com/paulboal/hds5210-2023/main/week06/allergies.json) and then upload it into Google Colab to run these examples. It is taken from sample data provided by the EHR vendor, Epic, here: https://open.epic.com/Clinical/Allergy

Take some time to look at the structure of the file.  You can open it directly in Jupyter by clicking the _Home_ icon, then the _from_instructor_ folder, and then the _data_ folder.

Within the file, you'll see that it is a dictionary with many items in it.  One of those items is called `entry` and that item is a list of things.  You can tell that because the item name is immediately followed by an opening square bracket, signifying the start of a list.  It's line 11 of the file: `  "entry": [`

Write a function named `allergy_count(json_file)` that takes as one parameter the name of the JSON file and returns an integer number of entries in that file.  Your function should open the file, read the json into a Python object, and return how many items there are in the list of `entry`s.

In [52]:
import json
ALLERGIES_FILE="allergies.json"

In [53]:
# Put your solution here
def allergy_count(json_file):
  try:
      with open(json_file, 'r') as file:
        data = json.load(file)
        if "entry" in data and isinstance(data["entry"], list):
            return len(data["entry"])
        else:
            return 0
  except FileNotFoundError:
      return 0

ALLERGIES_FILE = 'allergies.json'

In [54]:
allergy_count(ALLERGIES_FILE)

4

In [55]:
assert type(allergy_count(ALLERGIES_FILE)) == int
assert allergy_count(ALLERGIES_FILE) == 4

### 30.2 Number of Patients

If you dig a little bit deaper into this list of allergies, you'll see that each result has a patient associated with it.  Create a funcation called `patient_count(json_file)` that will count how many unique patients we have in this JSON structure.  

In [56]:
# Put your solution here
def patient_count(json_file):
  allegeries = json.load(open(json_file))
  patients = []
  for allegy in allegeries.get('entry'):
    patient = allegy.get('resource').get('patient')
    #appeding into a list avoiding the one already in the list
    if patient not in patients:
      patients.append(patient)
  return len(patients)

In [57]:
patient_count(ALLERGIES_FILE)

2

In [58]:
assert type(patient_count(ALLERGIES_FILE)) == int
assert patient_count(ALLERGIES_FILE) == 2

### 30.3 How Many Allergies per Patient

Although each entry is a separate allergy, several of them are for the same patient.  Write a function called `allergy_per_patient(json_file)` that counts up how many allergies each patient has.


In [59]:
def allergy_per_patient(json_file):
  allegeries = json.load(open(json_file))
  patient_allergies = {}
  for allegy in allegeries.get('entry'):
    patient = allegy.get('resource').get('patient').get('display')
    #appeding into a list avoiding the one already in the list dispalying their names and returning the number
    if patient:
      if patient in patient_allergies:
        patient_allergies[patient] += 1
      else:
        patient_allergies[patient] = 1
  return patient_allergies

In [60]:
allergy_per_patient(ALLERGIES_FILE)

{'Jason Argonaut': 3, 'Paul Boal': 1}

In [61]:
assert type(allergy_per_patient(ALLERGIES_FILE)) == dict
assert allergy_per_patient(ALLERGIES_FILE) == {'Paul Boal': 1, 'Jason Argonaut': 3}

### 30.4 Patient Allergies and Reaction

You'll see in the file that each of the items in the `entry` list have several other attributes including a patient name, substance text representation, and a reaction manifestation.  Create a function named `allergy_list(json_file)` that will create an output list that has patient name, allergy, and reaction for each `entry`.  The actual result you should get will be:

```python
[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis']]
```

You'll notice that the reaction and the manifestation of that action are lists.  You only need to capture the first reaction and the first manifestation of the action.  That is, if there is a list of things, just output the first one.

In [62]:
# Put your solution here
def allergy_list(json_file):
  allegeries = json.load(open(json_file))
  allergy_data = []
  for allegy in allegeries.get('entry'):
    patient_name = allegy.get('resource').get('patient').get('display')
    patient_sub = allegy.get('resource').get('substance').get('text')
    #it is a list so fetched with index
    patient_reaction = allegy.get('resource').get('reaction')[0]['manifestation'][0]['text']
    allergy_data.append([patient_name, patient_sub, patient_reaction])
  return allergy_data

In [63]:
allergy_list(ALLERGIES_FILE)

[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising']]

In [64]:
assert allergy_list(ALLERGIES_FILE) == [['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising']]


### 30.5 Allergy Reaction

Write a function called `allergy_reaction(json_file,patient,substance)` that takes three parameter and returns the reaction that will happen if the patient takes the specified substance.  You can solve this, in part, by calling your `allergy_list` function inside your new `allergy_reaction` function.

If the substance is not found in the allergy list, the function should return None.

In [65]:
# Put your solution here
def allergy_reaction(json_file, patient, substance):
  allergy_l = allergy_list(json_file)
  for entry in allergy_l:
    #assigning values
    patient_name, patient_subs, patient_reaction = entry
    if patient_name == patient and patient_subs == substance:
      return patient_reaction

In [66]:
allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G')

'Hives'

In [67]:
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G') == 'Hives'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS') == 'Itching'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'STRAWBERRY') == 'Anaphylaxis'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN') == None
assert allergy_reaction(ALLERGIES_FILE, 'Paul Boal', 'PENICILLIN G') == 'Bruising'

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to save this notebook file back to GitHub.  To do that in Google Colab:
1. File -> Save a Copy in GitHub
2. Make sure your HDS5210 repository is selected
3. Make sure the file name includes the week number like this: `week06/week06_assignment_2.ipynb`
4. Add a commit message that means something

**Be sure week names are lowercase and use a two digit week number!!**

**Be sure you use the same file name provided by the instructor!!**

